# Final Project Data Collection

In [67]:
import numpy as np
import pandas as pd

In [5]:
!aws s3 ls

2020-11-14 19:35:38 final-blurred-photos
2020-11-14 19:24:41 final-normal-photos
2020-10-22 22:05:07 network-bucket-nm
2020-10-20 22:24:15 source-image-350-nm
2020-10-20 22:28:56 target-image-nm


## Using Rekognition on Images

In [14]:
import boto3

In [15]:
client = boto3.client('rekognition')

### normal photos runthrough

In [29]:
normal = []
num = list(range(31))
for i in num:
    string = 'portrait_'+str(i)+'.jpg'
    normal.append(string)
normal = normal[1:]

In [36]:
normal_image_dicts = []
for image in normal:
    Dict = client.detect_faces(Attributes=["ALL"],Image={'S3Object':{'Bucket':'final-normal-photos','Name':image}})
    normal_image_dicts.append(Dict['FaceDetails'])

In [71]:
count = 1
metadata_normal = []
for image in normal_image_dicts:
    name = 'portrait_'+str(count)+'.jpg'
    count +=1
    for Dict in image:
        age = Dict['AgeRange']
        ageLow = age['Low']
        ageHigh = age['High']
        
        gender = Dict['Gender']
        genderValue = gender['Value']
        genderScore = np.round(gender['Confidence'],2)
        
        emotion = Dict['Emotions'][0]
        emotionType = emotion['Type']
        emotionScore = np.round(emotion['Confidence'],2)
        
        atrs = [name,ageLow,ageHigh,genderValue,genderScore,emotionType,emotionScore]
        metadata_normal.append(atrs)
metadata_normal[0]

['portrait_1.jpg', 18, 30, 'Female', 92.99, 'CALM', 63.34]

In [72]:
normal_df = pd.DataFrame(np.asarray(metadata_normal),columns=['filename','ageLow','ageHigh','gender','genderConfidence','emotion','emotionConfidence'])

In [73]:
normal_df.head()

,filename,ageLow,ageHigh,gender,genderConfidence,emotion,emotionConfidence
0,portrait_1.jpg,18,30,Female,92.99,CALM,63.34
1,portrait_2.jpg,13,25,Female,99.98,HAPPY,99.99
2,portrait_3.jpg,22,34,Male,98.97,HAPPY,86.23
3,portrait_4.jpg,21,33,Male,99.89,CALM,72.04
4,portrait_5.jpg,22,34,Female,99.58,CALM,50.5


In [77]:
#normal_df.to_csv('normal-images.csv')

### blurred photos runthrough

In [26]:
blurred = []
num = list(range(31))
for i in num:
    string = 'portrait_'+str(i)+'_blurred.jpg'
    blurred.append(string)
blurred = blurred[1:]

In [64]:
blurred_image_dicts = []
for image in blurred:
    Dict = client.detect_faces(Attributes=["ALL"],Image={'S3Object':{'Bucket':'final-blurred-photos','Name':image}})
    blurred_image_dicts.append(Dict['FaceDetails'])

In [74]:
count = 1
metadata_blurred = []
for image in blurred_image_dicts:
    name = 'portrait_'+str(count)+'_blurred.jpg'
    count +=1
    for Dict in image:
        age = Dict['AgeRange']
        ageLow = age['Low']
        ageHigh = age['High']
        
        gender = Dict['Gender']
        genderValue = gender['Value']
        genderScore = np.round(gender['Confidence'],2)
        
        emotion = Dict['Emotions'][0]
        emotionType = emotion['Type']
        emotionScore = np.round(emotion['Confidence'],2)
        
        atrs = [name,ageLow,ageHigh,genderValue,genderScore,emotionType,emotionScore]
        metadata_blurred.append(atrs)
metadata_blurred[0]

['portrait_1_blurred.jpg', 13, 23, 'Female', 99.63, 'CALM', 84.28]

In [75]:
blurred_df = pd.DataFrame(np.asarray(metadata_blurred),columns=['filename','ageLow','ageHigh','gender','genderConfidence','emotion','emotionConfidence'])

In [76]:
blurred_df.head()

,filename,ageLow,ageHigh,gender,genderConfidence,emotion,emotionConfidence
0,portrait_1_blurred.jpg,13,23,Female,99.63,CALM,84.28
1,portrait_2_blurred.jpg,19,31,Female,99.98,HAPPY,99.99
2,portrait_3_blurred.jpg,23,37,Male,97.53,HAPPY,96.63
3,portrait_4_blurred.jpg,20,32,Male,99.65,CALM,94.4
4,portrait_5_blurred.jpg,23,35,Female,99.37,CALM,54.59


In [78]:
#blurred_df.to_csv('blurred-images.csv')

In [84]:
blurred_df.rename(columns={'filename':'b_filename','ageLow':'b_ageLow','ageHigh':'b_ageHigh',
                          'gender':'b_gender','genderConfidence':'b_genderConf','emotion':'b_emotion',
                          'emotionConfidence':'b_emotionConf'},inplace=True)

In [85]:
df = pd.concat([normal_df,blurred_df],axis=1)

In [86]:
df.head()

,filename,ageLow,ageHigh,gender,genderConfidence,emotion,emotionConfidence,b_filename,b_ageLow,b_ageHigh,b_gender,b_genderConf,b_emotion,b_emotionConf
0,portrait_1.jpg,18,30,Female,92.99,CALM,63.34,portrait_1_blurred.jpg,13,23,Female,99.63,CALM,84.28
1,portrait_2.jpg,13,25,Female,99.98,HAPPY,99.99,portrait_2_blurred.jpg,19,31,Female,99.98,HAPPY,99.99
2,portrait_3.jpg,22,34,Male,98.97,HAPPY,86.23,portrait_3_blurred.jpg,23,37,Male,97.53,HAPPY,96.63
3,portrait_4.jpg,21,33,Male,99.89,CALM,72.04,portrait_4_blurred.jpg,20,32,Male,99.65,CALM,94.4
4,portrait_5.jpg,22,34,Female,99.58,CALM,50.5,portrait_5_blurred.jpg,23,35,Female,99.37,CALM,54.59


In [87]:
#df.to_csv('final-data.csv')